In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import phik
from phik.report import plot_correlation_matrix
from phik import report

The key aspect of ride-hailing is **upfront pricing**, which works the following way. 
*   First, it **predicts the price** for a ride **based on** predicted distance and time. This price is what you see on the screen of the phone before ordering a ride. 
*   Second, if **the metered price** based on actual distance and time **differs** a lot **from the predicted one**, the upfront price switches to the metered price.'A lot' means by more than 20%. 


---
For example, suppose you want to make a ride that upfront price predicts to cost 5 euros. 
If the **metered price is between 4 and 6 euros** - the rider pays 5 euros, otherwise the metered price.


---
We would like to **improve the upfront pricing precision**. Kindly analyze the data and **identify top opportunities** for that. Could you name the top one opportunity? 



In [28]:
df = pd.read_excel('Test.csv.xlsx')
print(df.shape)
df.head(3)

(4943, 26)


,order_id_new,order_try_id_new,calc_created,metered_price,upfront_price,distance,duration,gps_confidence,entered_by,b_state,dest_change_number,prediction_price_type,predicted_distance,predicted_duration,change_reason_pricing,ticket_id_new,device_token,rider_app_version,order_state,order_try_state,driver_app_version,driver_device_uid_new,device_name,eu_indicator,overpaid_ride_ticket,fraud_score
0,22.0,22.0,2020-02-02 03:37:30.573,4.04,10.0,2839.0,700.0,1.0,client,finished,1.0,upfront,13384.0,1091.0,NaN,1376.0,NaN,CI.4.17,finished,finished,DA.4.37,1596.0,Xiaomi Redmi 6,1.0,0.0,-1383.0
1,618.0,618.0,2020-02-08 02:26:18.924,6.09,3.6,5698.0,493.0,1.0,client,finished,1.0,upfront,2286.0,360.0,NaN,2035.0,NaN,CA.5.43,finished,finished,DA.4.39,1578.0,Samsung SM-G965F,1.0,0.0,NaN
2,657.0,657.0,2020-02-08 11:50:35.423,4.32,3.5,4426.0,695.0,1.0,client,finished,1.0,upfront,4101.0,433.0,NaN,2222.0,NaN,CA.5.43,finished,finished,DA.4.37,951.0,Samsung SM-A530F,1.0,0.0,-166.0


In [29]:
# % of null values in columns   
round(df.isna().mean().sort_values(ascending=False)*100).head(10)

device_token             100.0
change_reason_pricing     94.0
fraud_score               56.0
upfront_price             31.0
prediction_price_type      0.0
predicted_distance         0.0
predicted_duration         0.0
metered_price              0.0
rider_app_version          0.0
gps_confidence             0.0
dtype: float64

In [30]:
# consider the info above let's get rid of 'device_token' and 'change_reason_pricing' columns
df = df.drop(columns =['device_token', 'change_reason_pricing'])

df.drop_duplicates().shape, df.shape

((4943, 24), (4943, 24))

In [31]:
df['prediction_price_type'].value_counts() / df.shape[0]

upfront                        0.694315
prediction                     0.258750
upfront_destination_changed    0.042080
upfront_waypoint_changed       0.000809
Name: prediction_price_type, dtype: float64

in 70% cases of the data the 'upfront' prediction_price_type was applied.

### Let see correlations of the values when metered_price differs from the upfront_price by 20% to other columns



In [32]:
df =df[~df.upfront_price.isna()].copy() #consider only nonull values
df_next =df[df.upfront_price.isna()].copy()

In [33]:
df['price_diff'] = abs(df['metered_price'] - df['upfront_price']) /df['metered_price']

###! diff_more_20 - when the metered price differs from the upfront_price by 20%
df['diff_more_20'] = (df['price_diff'] >= 0.20) * 1 

In [34]:
df.diff_more_20.value_counts() # differs by 20% in 1364 rows

0    2045
1    1364
Name: diff_more_20, dtype: int64

In [35]:
df['diff_more_20'].mean() # differs by 20% in 40% of non null upfront prices

0.4001173364623057

In [36]:
import warnings
warnings.filterwarnings('ignore')

corr_matrix = df.phik_matrix()
corr_matrix[['diff_more_20']].sort_values(by=['diff_more_20'],ascending=False)\
                                                    .style.background_gradient(cmap='RdPu')

interval columns not set, guessing: ['order_id_new', 'order_try_id_new', 'metered_price', 'upfront_price', 'distance', 'duration', 'gps_confidence', 'dest_change_number', 'predicted_distance', 'predicted_duration', 'ticket_id_new', 'driver_device_uid_new', 'eu_indicator', 'overpaid_ride_ticket', 'fraud_score', 'price_diff', 'diff_more_20']


,diff_more_20
diff_more_20,1.000000
calc_created,1.000000
device_name,0.429982
gps_confidence,0.323787
duration,0.314773
eu_indicator,0.296276
metered_price,0.224632
driver_app_version,0.178249
order_try_id_new,0.162480
order_id_new,0.162480


## Top opportunity 1

most correlated (by 0.429982) columns is null **device_name**, lets see more detailed 

In [38]:
df.groupby('device_name')['diff_more_20'].agg(['count','mean']).sort_values('mean',ascending=False).head(10)

,count,mean
device_name,,
Samsung SM-N950U1,2,1.0
Samsung SM-N910V,2,1.0
Samsung SM-J415F,2,1.0
HUAWEI TRT-L21A,2,1.0
Xiaomi Redmi Note 5A,1,1.0
SHARP SH-02J,3,1.0
TECNO W5 Lite,5,1.0
TECNO W4,2,1.0
Samsung SM-J701F,1,1.0


In [39]:
df.device_name.str.split().str.get(0).str.replace('\d+', '', regex=True).unique()

array(['Xiaomi', 'Samsung', 'Itel', 'HUAWEI', 'TECNO', 'iPhone,',
       'TECNO-J', 'HMD', 'BullittGroupLimited', 'Infinix', 'Asus',
       'INFINIX', 'LENOVO', 'Sony', 'LGE', 'Wheatek', 'HighScreen',
       'Motorola', 'WIKO', 'ZTE', 'Alps', 'KONKA', 'BLU', 'Foxconn',
       'TECNO-L', 'OnePlus', 'Blackview', 'Google', 'OPPO', 'ITEL',
       'COSMO', 'SHARP', 'TCL', 'CUBOT_P', 'HTC', 'FUJITSU', 'INFINIX-X',
       'TECNO_W', 'Welcome', 'Ulefone', 'FISE', 'TECNO-W', 'JUST',
       'DOOGEE', 'DIGMA', 'TECNO-C', 'TP-Link', 'OUKITEL', 'Cat',
       'Prestigio', 'HTC_Du', 'Coolpad', 'Meizu', 'Lava'], dtype=object)

In [40]:
df.device_name = df.device_name.str.split().str.get(0).str.replace('\d+', '', regex=True)
df.loc[df.device_name.str.contains('TECNO'),'device_name'] = 'TECNO MOBILE'

In [41]:
df.groupby('device_name')['diff_more_20'].agg(['count','mean']).sort_values('count',ascending=False).head(15)

,count,mean
device_name,,
Samsung,1309,0.355997
HUAWEI,630,0.365079
TECNO MOBILE,402,0.582090
"iPhone,",357,0.372549
Xiaomi,236,0.372881
HMD,92,0.434783
INFINIX,65,0.600000
Itel,41,0.560976
LGE,37,0.297297


### Conclusion
---

in the table above a list of devices that makes worse the upfront pricing precision. Recommendation is to change type of mobile.

---



## Top opportunity 2 

second most correlated (by 0.323787) columns is null **gps_confidence**, lets see more detailed  

In [42]:
df.groupby('gps_confidence')['diff_more_20'].agg(['count','mean'])\
  .sort_values('count', ascending=False).head(10)

,count,mean
gps_confidence,,
1.0,3037,0.363846
0.0,372,0.696237



### Conclusion

---

from the aggregations **above** we can suppose, that the upfront_price precision can be improved by gps tracking device.

---



## Top opportunity 3 

third most correlated (by0.314773) columns is null **duration**, lets see more detailed 

In [43]:
df.groupby('duration')['diff_more_20'].agg(['count','mean'])\
  .sort_values('count', ascending=False).head(5)

,count,mean
duration,,
0.0,20,0.950000
342.0,9,0.111111
866.0,9,0.444444
793.0,8,0.500000
1111.0,8,1.000000


In [44]:
# lets segmentate duration values by the frequency of those values for 6 bins.

df['duration_freq'] = pd.qcut(df['duration'], 6)
df.groupby('duration_freq')['diff_more_20'].agg(['count','mean'])\
                                    .sort_values('mean', ascending=False)

,count,mean
duration_freq,,
"(1863.0, 18734.0]",568,0.637324
"(-0.001, 419.0]",569,0.421793
"(895.0, 1213.0]",566,0.374558
"(1213.0, 1863.0]",568,0.373239
"(649.0, 895.0]",569,0.339192
"(419.0, 649.0]",569,0.254833



### Conclusion

---

here we can conclude that for long distances the upfront pricing doesn't work well. It can be considered in ML modelling.

---



## Top opportunity 4 

next  correlated (by 0.296276) columns is null **eu_indicator**, lets see more detailed 

In [45]:
df.groupby('eu_indicator')['diff_more_20'].agg(['count','mean'])

,count,mean
eu_indicator,,
0.0,882,0.560091
1.0,2527,0.344282



### Conclusion

---

the upfront_price precision can be improved by improving service in non Europian Union locations. 

---



## Top opportunity 5 

next  correlated (by 	0.224632
) columns is null **metered_price**, lets see more detailed 

In [46]:
# lets segmentate duration values by the frequency of those values for 6 bins.

df['metered_price_freq'] = pd.qcut(df['metered_price'], 6)
df.groupby('metered_price_freq')['diff_more_20'].agg(['count','mean'])\
                                    .sort_values('count', ascending=False)

,count,mean
metered_price_freq,,
"(1.999, 3.73]",572,0.328671
"(3.73, 5.2]",568,0.286972
"(7.37, 12.39]",568,0.382042
"(12.39, 7940.22]",568,0.433099
"(7940.22, 194483.52]",568,0.607394
"(5.2, 7.37]",565,0.362832



### Conclusion

---

the metered_price in range (7940.22 - 194483.52) affects on upfront pricing significantly, the metered_price's mathematics should be reconsidered in such ranges. Assume, that it consists of two main variable distance and duration, which affects mostly.

---



## Top opportunity 6 

next less correlated (by 	0.178249
) columns is null **driver_app_version**, lets see more detailed 

In [47]:
df.groupby('driver_app_version')['diff_more_20'].agg(['count','mean'])\
  .sort_values('count',ascending=False).head(7)

,count,mean
driver_app_version,,
DA.4.39,1037,0.368370
DA.4.37,787,0.419314
DA.4.42,458,0.373362
DA.4.31,227,0.396476
DA.4.34,208,0.447115
DA.4.41,123,0.284553
DI.3.33,96,0.333333



### Conclusion

---

the driver_app_version also affects on upfront pricing. The suggestion is to make a alertion in the system for a driver to update app version.

---

